In [3]:
from table_item import TableItem
from openai import OpenAI

class TableItem:
    """
    TableItem class to represent a column or measure in a table.
    """

    def __init__(self, table_item_id: str, name: str, table_item_type: str = 'column', data_type: str = None, format_string: str = None,
                 display_folder: str = None, is_hidden: bool = False, expression: list = None):
        """
        Constructor of the class
        :param table_item_id: id of the table item
        :param name: name of the table item
        :param table_item_type: type of the table item, can be 'column', 'calculated' or 'measure'
        :param data_type: type of the data, e.g. 'String', 'Integer', 'date', etc.
        :param format_string: the format method for measures
        :param display_folder: the folder where the table item is displayed
        :param is_hidden: if the table item is hidden
        :param expression: the definition of the measure or calculated column. It is a list of string lines.
        """
        self.table_item_id: str = table_item_id
        self.name: str = name
        self.table_item_type: str = table_item_type
        self.data_type: str = data_type
        self.format_string: str = format_string
        self.display_folder: str = display_folder
        self.is_hidden: bool = is_hidden
        self.expression: list = expression

    def __str__(self):
        """
        Method to return a string representation of the table item
        :return: string
        """
        result = ''
        result += f'Name: {self.name}\n'
        result += f'Type: {self.table_item_type}\n'
        result += f'Data Type: {self.data_type}\n'
        result += f'Format String: {self.format_string}\n'
        result += f'Display Folder: {self.display_folder}\n'
        result += f'Is Hidden: {self.is_hidden}\n'
        result += f'Expression: {self.expression}\n'
        return result


In [6]:
table_item = TableItem('1', 'Variação CPL', 'measure')
table_item.expression = [
'var divisao =DIVIDE([CPL Ult 7d], [CPL Ult 8 - 14d])',
'return IF(ISBLANK([CPL Ult 7d]) && not ISBLANK([CPL Ult 8 - 14d]), 999, IF(not ISBLANK([CPL Ult 7d]) && ISBLANK([CPL Ult 8 - 14d]), 1, IF(ISBLANK([CPL Ult 7d]) && ISBLANK([CPL Ult 8 - 14d]), 0, divisao-1)))'
]

    

In [19]:
with open ('openai-key.txt', 'r') as file:
    client = OpenAI(api_key=file.read())

expression = '\n'.join(table_item.expression)
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system",
         "content": "You respond objectively about dax measures"},
        {"role": "user", 
         "content": f"Explique resumidamente a medida '{table_item.name}': `{expression}`"}
    ],
    max_tokens=200
)

In [20]:
m = completion.choices[0].message.content

In [21]:
print(m)
m

A medida 'Variação CPL' é calculada dividindo o valor de CPL (custo por lead) nos últimos 7 dias pelo valor de CPL nos dias 8 a 14 atrás. A fórmula lida com diferentes cenários de valores em branco: se o CPL dos últimos 7 dias estiver em branco e o CPL dos dias 8 a 14 dias não estiver em branco, retorna 999; se o CPL dos últimos 7 dias não estiver em branco e o CPL dos dias 8 a 14 dias estiver em branco, retorna 1; se ambos os valores estiverem em branco, retorna 0; caso contrário, retorna a divisão dos valores menos 1. Essa medida ajuda a entender a variação entre o CPL recente e o CPL mais antigo em um determinado período.


"A medida 'Variação CPL' é calculada dividindo o valor de CPL (custo por lead) nos últimos 7 dias pelo valor de CPL nos dias 8 a 14 atrás. A fórmula lida com diferentes cenários de valores em branco: se o CPL dos últimos 7 dias estiver em branco e o CPL dos dias 8 a 14 dias não estiver em branco, retorna 999; se o CPL dos últimos 7 dias não estiver em branco e o CPL dos dias 8 a 14 dias estiver em branco, retorna 1; se ambos os valores estiverem em branco, retorna 0; caso contrário, retorna a divisão dos valores menos 1. Essa medida ajuda a entender a variação entre o CPL recente e o CPL mais antigo em um determinado período."